## Part 1 Wikipedia Data Set

In [2]:
#Required Imports
import numpy as np 
import pandas as pd 
import requests 
print('Import Complete')

Import Complete


In [3]:
#Required Imports
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
print('Import Complete')

Import Complete


In [4]:
#Obtaining data from Wikipedia
Wiki = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
Bea_soup = BeautifulSoup(Wiki.text, 'lxml')

#Getting the data from HTML and converting into a table using Beautiful Soup

data = []
columns = []
table = Bea_soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #Making it known that the first row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#Pandas dataframe conversion
toto_df = pd.DataFrame(data = data,columns = columns)

In [10]:
toto_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
toto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code     180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [6]:
#Data Cleanup
#Removing Boroughs that are listed as 'not assigned'

toto_df = toto_df[toto_df['Borough'] != 'Not assigned']


In [7]:
toto_df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
#Removing Duplicates from the dataset
toto_df['Borough']= toto_df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [9]:
#Dropped Duplicates
toto_df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,NorthYork,Parkwoods
3,M4A,NorthYork,Victoria Village
4,M5A,DowntownToronto,"Regent Park, Harbourfront"
5,M6A,NorthYork,"Lawrence Manor, Lawrence Heights"
6,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,NorthYork,Don Mills
12,M4B,EastYork,"Parkview Hill, Woodbine Gardens"
13,M5B,DowntownToronto,"Garden District, Ryerson"


In [10]:
#Updated DF info
toto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


In [11]:
toto_df.shape

(103, 3)

## Part 2 Obtaining Latitude and Longitude

In [13]:
#Identify Location in Toronto Test
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 124974741, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [16]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates are downloaded')
torogla = pd.read_csv('toronto_coordinates.csv')

Coordinates are downloaded


In [18]:
print(torogla.shape)
print(torogla.info)
torogla.head(5)

(103, 3)
<bound method DataFrame.info of     Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24       

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
# Merger of Dataframes

ronto_temp = toto_df.set_index('Postal Code')
kors_temp = torogla.set_index('Postal Code')
ronto_coors_df = pd.concat([ronto_temp, kors_temp], axis=1, join='inner')

ronto_coors_df.index.name = 'Postal Code'
ronto_coors_df.reset_index(inplace=True)

print(ronto_coors_df.shape)
ronto_coors_df.head(15)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,Victoria Village,43.725882,-79.315572
2,M5A,DowntownToronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,NorthYork,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,NorthYork,Don Mills,43.745906,-79.352188
8,M4B,EastYork,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,DowntownToronto,"Garden District, Ryerson",43.657162,-79.378937
